<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/2_3_SPRINTCHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 2 Sprint Challenge 3

## Logistic Regression and Beyond

In this sprint challenge you will fit a logistic regression modeling the probability of an adult having an income above 50K. The dataset is available at UCI:

https://archive.ics.uci.edu/ml/datasets/adult

Your goal is to:

1. Load, validate, and clean/prepare the data.
2. Fit a logistic regression model
3. Answer questions based on the results (as well as a few extra questions about the other modules)

Don't let the perfect be the enemy of the good! Manage your time, and make sure to get to all parts. If you get stuck wrestling with the data, simplify it (if necessary, drop features or rows) so you're able to move on. If you have time at the end, you can go back and try to fix/improve.

### Hints

It has a variety of features - some are continuous, but many are categorical. You may find [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) (a method to one-hot encode) helpful!

The features have dramatically different ranges. You may find [sklearn.preprocessing.minmax_scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html#sklearn.preprocessing.minmax_scale) helpful!

## Part 1 - Load, validate, and prepare data

The data is available at: https://archive.ics.uci.edu/ml/datasets/adult

Load it, name the columns, and make sure that you've loaded the data successfully. Note that missing values for categorical variables can essentially be considered another category ("unknown"), and may not need to be dropped.

You should also prepare the data for logistic regression - one-hot encode categorical features as appropriate.

In [143]:
'''
>50K, <=50K.

age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
fnlwgt: continuous.
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
education-num: continuous.
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.
capital-gain: continuous.
capital-loss: continuous.
hours-per-week: continuous.
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
'''
# fnlwgt appears to be... nothing...
FEATURE_names = ['age', 'workclass', 'fnlwgt','education', 'education_num', 
                  'marital_status', 'occupation', 'relationship', 'race', 'sex', 
                  'capital_gain', 'capital_loss', 'hours_per_week', 
                  'native_country', 'TARGET']

train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

df_ = pd.read_csv(train_url, header=None, names=FEATURE_names, skipinitialspace=True)

side_hustle = {'?': 'Side_hustle'}
df_.workclass = df_.workclass.replace(side_hustle)

edu_replace = {'Preschool': 0, '1st-4th': 1, '5th-6th': 2, '7th-8th': 3, 
               '9th': 4, '10th': 5, '11th': 6, '12th': 7, 'HS-grad': 8, 
               'Some-college': 9, 'Assoc-voc': 10, 'Assoc-acdm': 10, 
               'Prof-school': 11, 'Bachelors': 13, 'Masters': 14, 
               'Doctorate': 15} # THIS WAS A WASTE OF TIME because education_num does this for me.
assert len(edu_replace.keys())==len(df_.education.value_counts().index)

df_.occupation = df_.occupation.replace(side_hustle)

df_.native_country = df_.native_country.replace({'?': 'Unknown'})#.apply(lambda s: s + '_ORIGIN')

# for capital_gain and capital_loss, i mostly care about the difference between 0 and 0+. 
# i'll pronounce this. 
df_['log_capital_gain'] = np.log(df_.capital_gain + 1)
df_['log_capital_loss'] = np.log(df_.capital_loss + 1)

df_.TARGET = df_.TARGET.replace({'<=50K': 0, '>50K': 1})

to_onehot = ['workclass', 'marital_status', 'occupation', 'relationship', 
             'race', 'sex', 'native_country']
# drop after onehotting. 

to_drop = ['education', 'capital_gain', 'capital_loss'] + to_onehot

df_.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,TARGET,log_capital_gain,log_capital_loss
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,7.684784,0.0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0,0.000000,0.0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0,0.000000,0.0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0,0.000000,0.0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0,0.000000,0.0


In [0]:
df_test_ = pd.read_csv(test_url, header=None, names=FEATURE_names, skiprows=1, skipinitialspace=True)

#df_test_.head()

In [146]:
df = pd.concat([df_.drop(to_drop, axis=1), pd.get_dummies(df_[to_onehot])], axis=1, join='outer')

f = lambda s: s.replace('-', '_')

df.columns = [f(x) for x in df.columns]

assert df.shape[0]==df_.shape[0]
#assert df_.shape[1]-3+pd.get_dummies(df_[to_onehot]).shape[1]==df.shape[1] # There's an extra `+3` here because of the dropped columns
df.shape

(32561, 93)

## Part 2 - Fit and present a Logistic Regression

Your data should now be in a state to fit a logistic regression. Use scikit-learn, define your `X` (independent variable) and `y`, and fit a model.

Then, present results - display coefficients in as interpretible a way as you can (hint - scaling the numeric features will help, as it will at least make coefficients more comparable to each other). If you find it helpful for interpretation, you can also generate predictions for cases (like our 5 year old rich kid on the Titanic) or make visualizations - but the goal is your exploration to be able to answer the question, not any particular plot (i.e. don't worry about polishing it).

It is *optional* to use `train_test_split` or validate your model more generally - that is not the core focus for this week. So, it is suggested you focus on fitting a model first, and if you have time at the end you can do further validation.

In [151]:
dependent = 'TARGET'
X = df.drop(dependent, axis=1)
y = df[dependent]

X_train, X_test, y_train, y_test = train_test_split(X, y)

m = LogisticRegression(solver='lbfgs').fit(X_train,y_train)

m.predict_proba(X_test)

array([[0.677011  , 0.322989  ],
       [0.60249545, 0.39750455],
       [0.95164255, 0.04835745],
       ...,
       [0.84729003, 0.15270997],
       [0.64017829, 0.35982171],
       [0.80518444, 0.19481556]])

In [140]:
'feis-fkdl'.replace('-', '_')

'feis_fkdl'

## Part 3 - Analysis, Interpretation, and Questions

### Based on your above model, answer the following questions

1. What are 3 features positively correlated with income above 50k?
2. What are 3 features negatively correlated with income above 50k?
3. Overall, how well does the model explain the data and what insights do you derive from it?

*These answers count* - that is, make sure to spend some time on them, connecting to your analysis above. There is no single right answer, but as long as you support your reasoning with evidence you are on the right track.

Note - scikit-learn logistic regression does *not* automatically perform a hypothesis test on coefficients. That is OK - if you scale the data they are more comparable in weight.

### Match the following situation descriptions with the model most appropriate to addressing them

In addition to logistic regression, a number of other approaches were covered this week. Pair them with the situations they are most appropriate for, and briefly explain why.

Situations:
1. You are given data on academic performance of primary school students, and asked to fit a model to help predict "at-risk" students who are likely to receive the bottom tier of grades.
2. You are studying tech companies and their patterns in releasing new products, and would like to be able to model and predict when a new product is likely to be launched.
3. You are working on modeling expected plant size and yield with a laboratory that is able to capture fantastically detailed physical data about plants, but only of a few dozen plants at a time.

Approaches:
1. Ridge Regression
2. Quantile Regression
3. Survival Analysis

**TODO - your answers!**